# Collating results from `comparison_runs.py`

Going to use this for both CS287H report and my paper. Might copy some portions from `collate_csvs.py`, which this notebook supersedes.

In [ ]:
%matplotlib inline
import collections
import colorsys
import glob
import os
import re
import sys

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sys.path.append('.')
from comparison_runs import EnvName
del sys.path[-1]

sns.set(context='paper', style='darkgrid')

In [ ]:
CSV_PATTERN = '../scratch/runs-1seed-svm-2020-05-11/run_*/eval-*.csv'

In [ ]:
csv_paths = glob.glob(os.path.expanduser(CSV_PATTERN))
print(f"Found {len(csv_paths)} CSV files")
loaded_csvs = [pd.read_csv(c) for c in csv_paths]
frame = pd.concat(loaded_csvs)
frame

Now going to add some extra data:

- Human-readable test variant names.
- String-formatted mean and median for the LaTeX tables.

In [ ]:
def to_variant(env_name):
    en = EnvName(env_name)
    return en.demo_test_spec.strip('-')

def to_prefix(env_name):
    en = EnvName(env_name)
    return en.name_prefix.strip('-')

frame['variant'] = frame['test_env'].map(to_variant)
frame['env_prefix'] = frame['demo_env'].map(to_prefix)

## Separate tables for each task

Here I'm going to create separate LaTeX tables for each task. Each row will be a method, and each column will be a variant, with cells showing both mean and standard deviation of performance. Should have the same set of columns for each method.

In [ ]:
VARIANT_SHORT_NAMES = collections.OrderedDict([
  ("Demo", "Demo"),
  ("TestJitter", "Jitter"),
  ("TestLayout", "Layout"),
  ("TestColour", "Colour"),
  ("TestShape", "Shape"),
  ("TestCountPlus", "CountPlus"),
  ("TestDynamics", "Dynamics"),
  ("TestAll", "All"),
])
VARIANT_ORDER = {
    env_name: index for index, env_name
    in enumerate(VARIANT_SHORT_NAMES.keys())
}
PROBLEM_PREFIX_ORDER = [
    'MoveToCorner',
    'MoveToRegion',
    'MatchRegions',
    'FindDupe',
    'ClusterColour',
    'ClusterType',
]

Going to do some sanity checks on the variants to make sure that none are missing from `VARIANT_SHORT_NAMES` or `VARIANT_ORDER`.

In [ ]:
all_variants = set(frame['variant'].unique())
missing = all_variants - VARIANT_ORDER.keys()
if missing:
    print(f"{len(missing)} variants are missing:")
    print(sorted(missing))
    assert False, 'figure out what is going wrong!'

In [ ]:
def convert_method_name(name):
    # TODO: fix my config file so that it includes human-readable names. That should avoid this problem :)
    
    # remove the "on-<task>" suffix
    suffix_re = re.compile(r'-on-(find-|cluster-|match-|move-|move-)[a-z-]*$')
    name = suffix_re.sub('', name)

    # figure out the algorithm prefix ("BC" or "GAIL")
    prefix_re = re.compile(r'(bc|gail)(-.*)?')
    match = prefix_re.fullmatch(name)
    if not match:
        return name  # not handled
    prefix, name = match.groups()
    name = name.strip('-') if name is not None else ''
    prefix = prefix.upper()
    
    # now handle the rest
    if name == 'mt':
        full_name = prefix + ' (MT)'
    elif name == '':
        full_name = prefix
    else:
        full_name = f'{prefix} ({name})'

    return full_name

In [ ]:
def lighten(rgba):
    assert len(rgba) == 4, len(rgba)
    rgb = rgba[:3]
    a = rgba = rgba[3]
    h, l, s = colorsys.rgb_to_hls(*rgb)
    l = 1 - (1 - l) * a
    rgb = colorsys.hls_to_rgb(h, l, s)
    return rgb

def assign_colours(values, legend=False):
    r"""Create the LaTeX \cellcolor commands necessary for a series of values.
    Values are assumed to be in [0,1]."""
    # (requires \usepackage[table]{xcolor}, per
    # https://tex.stackexchange.com/a/50351)
    values = np.asarray(values)
    values[values < 0] = 0
    values[values > 1] = 1
    cmap = plt.get_cmap('Blues')
    cols_rgba = cmap(values, alpha=0.2)
    cols_rgb = map(lighten, cols_rgba)
    if legend:
        # a crude LaTeX colour gradient
        return r'\!'.join(r'\crule{%.2f,%.2f,%.2f}' % spec for spec in cols_rgb)
    else:
        specs = [
            r'\cellcolor[rgb]{%.2f,%.2f,%.2f}' % tuple(spec)
            for spec in cols_rgb
        ]
    return specs

def compute_cell_contents(tab):
    (_, col_name), *_ = tab.columns.to_flat_index()
    mean_series = tab['mean_score'].squeeze()
    std_series = tab['std_score'].squeeze()
    cols = assign_colours(mean_series)
    result_list = [
        r'%s %.2f (%.2f)' % col_mu_std
        for col_mu_std in zip(cols, mean_series, std_series)
    ]
    # now construct a new frame with column name taken from 'tab', and indices
    # taken from 'mean_series' and 'std_series'
    contents = pd.Series(data=result_list, index=tab.index, name=col_name)
    return contents

print('Colour scale:', assign_colours(np.linspace(0, 1, 8), legend=True))
print('')

demo_envs = sorted(
    frame['demo_env'].unique(),
    key=lambda s: PROBLEM_PREFIX_ORDER.index(s.split('-')[0]))
out_lines = []
for demo_env in demo_envs:
    prefix = to_prefix(demo_env)
    subset = frame[frame['demo_env'] == demo_env]
    pivot = subset.pivot(index='latex_alg_name',
                         columns='variant',
                         values=['mean_score', 'std_score'])
    renamed_pivot = pivot.groupby(axis=1, level=1).apply(compute_cell_contents)
    for col_name in VARIANT_SHORT_NAMES:
        if col_name not in renamed_pivot.columns:
            renamed_pivot[col_name] = '-'
    sorted_pivot = renamed_pivot[list(VARIANT_SHORT_NAMES)]
    sorted_cols = [r'\textbf{' + VARIANT_SHORT_NAMES.get(c, c) + '}' for c in sorted_pivot.columns]
    sorted_pivot.columns = pd.MultiIndex.from_product([(f'\\textbf{{{prefix}}}', ), sorted_cols], names=('Task', 'Variant'))
    sorted_pivot.index = sorted_pivot.index.map(convert_method_name).rename("Method")
    sorted_pivot.sort_index(axis=0, inplace=True)
    latex_formatted = sorted_pivot.to_latex(
        # label=f'tab:res-{prefix.lower()}',
        column_format='l' + 'c' * len(sorted_cols),
        bold_rows=True,
        escape=False,
    ).replace('{l}', '{c}')
    print(latex_formatted)
    
    latex_lines = latex_formatted.splitlines()
    if not out_lines:
        # keep first few lines
        out_lines.extend(latex_lines[:2])
    out_lines.extend(latex_lines[2:-2])
    if demo_env == demo_envs[-1]:
        out_lines.extend(latex_lines[-2:])
    else:
        out_lines.extend(['', r'\midrule', ''])

print('\n\nFull result:\n')
print('\n'.join(out_lines))